In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [10]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", {"train": foo_data, "valid": foo_data}, foo_input, foo_head)
bar = Task("bar_task", {"train": bar_data, "valid": bar_data}, bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [19]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, device=-1, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.1, 
    progress_bar=True,
    log_unit="epochs",
    log_every=0.25,
    score_every=0.5,
    writer="tensorboard",
    checkpoint_best=True,
    checkpoint_metric="foo_task/valid/accuracy",
    checkpoint_metric_mode="max",
)

Beginning train loop.
Expecting a total of 5600 examples and 1400 batches per epoch from 2 tasks.


[0.25 epo]: TRAIN:[loss=0.028]
[0.50 epo]: TRAIN:[loss=0.195] VALID:[foo_task/accuracy=0.975, bar_task/accuracy=0.981]
Saving model at iteration 0.5 with best (max) score 0.975
[0.75 epo]: TRAIN:[loss=0.048]
[1.0 epo]: TRAIN:[loss=0.041] VALID:[foo_task/accuracy=0.982, bar_task/accuracy=1.000]
Saving model at iteration 1.0 with best (max) score 0.982



[1.25 epo]: TRAIN:[loss=0.008]
[1.50 epo]: TRAIN:[loss=0.021] VALID:[foo_task/accuracy=0.989, bar_task/accuracy=0.999]
Saving model at iteration 1.5 with best (max) score 0.989
[1.75 epo]: TRAIN:[loss=0.201]
[2.0 epo]: TRAIN:[loss=0.093] VALID:[foo_task/accuracy=0.996, bar_task/accuracy=0.999]
Saving model at iteration 2.0 with best (max) score 0.996



[2.25 epo]: TRAIN:[loss=0.029]
[2.50 epo]: TRAIN:[loss=0.073] VALID:[foo_task/accuracy=0.992, bar_task/accuracy=0.994]
[2.75 epo]: TRAIN:[loss=0.057]
[3.0 epo]: TRAIN:[loss=0.041] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.998]

Restoring best model from iteration 2.0 with score 0.996
Writing log to /Users/bradenjh/repos/metal/logs/2019_02_11/14_39_44/14_39_44.json
Finished Training
{'bar_task/valid/accuracy': 0.99875, 'foo_task/valid/accuracy': 0.99625}


In [17]:
for batch in foo.data_loaders["train"]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

{'foo_task': tensor([[ 64.8021, -66.2816],
        [-71.4408,  72.2064],
        [ 63.6610, -65.1231],
        [ -0.1811,  -0.2317]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor(0.1797, grad_fn=<NllLossBackward>)}
{'foo_task': tensor([[1.0000, 0.0000],
        [0.0000, 1.0000],
        [1.0000, 0.0000],
        [0.5126, 0.4874]])}


In [ ]:
next(model.parameters()).device